# Spectral resolution and subaperturing

In these notes, I'm looking at the effect that subaperturing has on the spectral resolving power for Lynx. Since most parts of the Lynx design are still in the air, there are a lot of assumptions in here, some of which might have a major influence on the result. This is a document in progress that must evolve with the evolving mission design.

Please contact me (Moritz) if you want to use any of the figures or plots below for presentations or further reseach and I will prepare the high-resolution figures for you or provide you with the datasets behind the plot in numeric form. Please do not copy and paste the figures here; they are intentionally made at a lower resolution to optimize the display on a webpage.

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
from __future__ import print_function, division
import sys
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.table import Table, join
import astropy.units as u
import marxs
from marxs import visualization

from marxs.source import PointSource, FixedPointing, JitterPointing
from marxs.analysis import resolvingpower_from_photonlist
from marxs.simulator import Sequence

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from marxslynx import lynx
from marxslynx.bendgratings import bend_gratings, chirp_flat_grating
from marxslynx.ralfgrating import facet_table, order_selector_Si, order_selector_SiPt
import marxslynx

In [ ]:
import marxs.version
print('MARXS version {} (commit hash: {} from {})'.format(marxs.version.version, marxs.version.githash, marxs.version.timestamp))

In [ ]:
import marxs.version
import marxslynx.version
print('MARXS version {} (commit hash: {} from {})'.format(marxs.version.version, marxs.version.githash, marxs.version.timestamp))
print('Lynx version {} (commit hash: {} from {})'.format(marxslynx.version.version, marxslynx.version.githash, marxslynx.version.timestamp))

In [ ]:
def run_monoenergetic_simulation(instrument, energy, n_photons=2e4):
    mysource = PointSource(coords=SkyCoord(0., 0., unit='deg'),
                           energy=energy,
                           flux=1.)
    fixedpointing = FixedPointing(coords=SkyCoord(0., 0., unit='deg'))
    photons = mysource.generate_photons(n_photons)
    photons = fixedpointing(photons)
    photons = instrument(photons)
    return photons[np.isfinite(photons['order'])]
    

## Zeroth order image

First, we look at a zeroth order image. That allows us to check that we got the mirror right. This simulation uses a simplified mirror model. It does not simulate individual mirror shells, but instead is treats the mirror as a continuum. Every photon hitting the mirror plane is redicted to the focal point assuming an "ideal" mirror. Then, additional scatter is added both in the plane of reflection and out of the plane of reflection. Typically, for an individual shell, the scatter is larger in the plane of reflection due to figure errors and the scattering by particulates.

In this case, we want to simulate a mirror where the PSF is domianted by the mis-pointing of individual mirror shells (which is the case where sub-aperturing is the least useful). Mis-pointing of the shells happens in both x and y direction, which, looking at a single photon, can be simulated by assuming a perfect mirror and scatter of the same magnitude in the plane of the scattering and out of the plane of the scattering. 
Note that for simulations of gratings it would be cheating to apply a blur in the focal plane to account for the PSF in the way that many simpler simulations programs do it, because we need to know the direction of the photon already when it hits the gratings, and not just in the focal plane.

In [ ]:
o7triplet = (21.8 * u.Angstrom).to(u.keV, equivalencies=u.spectral())
single_en = 0.6

In [ ]:
import copy 
conf_5050 = copy.copy(lynx.conf)
conf_5050['grating_size'] = np.array([50., 50.])
lynx5050 = lynx.PerfectLynx(conf=conf_5050)

facettab5050 = facet_table(lynx5050.elements[2])
facetpos5050 = np.asanyarray(lynx5050.elements[2].elem_pos)

p5050 = run_monoenergetic_simulation(lynx5050, single_en)
p5050 = join(p5050, facettab5050)

In [ ]:
lynxdef = lynx.PerfectLynx(conf=lynx.conf)
facettabdef = facet_table(lynxdef.elements[2])
facetposdef = np.asanyarray(lynxdef.elements[2].elem_pos)

pdef = run_monoenergetic_simulation(lynxdef, single_en)
pdef = join(pdef, facettabdef)

In [ ]:
import copy
conf_scat = copy.copy(lynx.conf)
conf_scat['inplanescatter'] = 3.5e-6
conf_scat['perpplanescatter'] = .7e-6
l_scat = Sequence(elements=[lynxdef.elements[0], marxslynx.mirror.MetaShell(conf_scat)] + lynxdef.elements[2:])
l_scat = lynx.PerfectLynx(conf=conf_scat)
p_scat = run_monoenergetic_simulation(l_scat, single_en)

p_scat = join(p_scat, facettabdef)

In [ ]:
#perfectlynx_b_scat = lynx.PerfectLynx(conf=conf_scat)
bend_gratings(conf_scat, l_scat.elements[2], r=9500)

pb_scat = run_monoenergetic_simulation(l_scat, single_en)
pb_scat = join(pb_scat, facettabdef)

In [ ]:
import numpy as np
from numpy.core.umath_tests import inner1d
from scipy.interpolate import RectBivariateSpline
from transforms3d.affines import decompose
import astropy.units as u

from marxs.math.geometry import Cylinder
from marxs.math.utils import h2e, norm_vector
from marxs.utils import generate_test_photons
from marxs.optics import CATGrating
from marxslynx.lynx import detcirc

def find_where_ref_ray_should_go(conf, order, wave):
    energy = wave.to(u.keV, equivalencies=u.spectral())
    rays = generate_test_photons(2)
    rays['energy'] = energy.value
    rays['pos'][:, 0] = 1e5

    def mock_order(x, y, z):
        return np.array([0, order]), np.ones(2)

    pos = conf['rowland'].solve_quartic(y=0, z=0, interval=[9e3, 1.2e4])
    test_grat = CATGrating(d=conf['grating_d'], order_selector=mock_order,
                            position=[pos, 0, 0],
                            orientation=conf['blazemat'] )
    rays = test_grat(rays)
    rays = detcirc(rays)
    return rays['pos'].data[0, :], rays['pos'].data[1, :]


def chirp_flat_grating(conf, gas, order, wave, n_points=[3, 3]):
    '''
    Parameters
    ----------
    wave : `astropy.quantity.Quantity`
    '''
    focalpoint, ref_point = find_where_ref_ray_should_go(conf, order, wave)
    pos_on_e = np.meshgrid(np.linspace(-1, 1, n_points[0]), np.linspace(-1, 1, n_points[1]))
    d_needed = np.zeros_like(pos_on_e[0])
    for e in gas.elements:
        l_x = np.linalg.norm(e.geometry['v_y'])
        l_y = np.linalg.norm(e.geometry['v_z'])
        for i in range(pos_on_e[0].shape[0]):
            for j in range(pos_on_e[0].shape[1]):
                positions = h2e(e.geometry['center']) + pos_on_e[0][i, j] * h2e(e.geometry['v_y']) + pos_on_e[1][i, j] * h2e(e.geometry['v_z'])
                vec_pos_foc = - positions[:2] + h2e(focalpoint)[:2]
                vec_pos_foc = vec_pos_foc / np.linalg.norm(vec_pos_foc)
                vec_pos_ref_point = - positions[:2] + h2e(ref_point)[:2]
                vec_pos_ref_point = vec_pos_ref_point / np.linalg.norm(vec_pos_ref_point)

                theta_needed = np.arccos(np.dot(vec_pos_foc, vec_pos_ref_point))
                d_needed[i, j] = np.abs(order) * wave.to(u.mm).value / np.sin(theta_needed)
        e._d = RectBivariateSpline(pos_on_e[0][0, :], pos_on_e[1][:, 0], d_needed)

def chirp_flat_grating2(conf, gas, order, wave):
    '''
    Parameters
    ----------
    wave : `astropy.quantity.Quantity`
    '''
    focalpoint, ref_point = find_where_ref_ray_should_go(conf, order, wave)
    for e in gas.elements:
        d_needed = np.zeros(3)
        l_x = np.linalg.norm(e.geometry['v_y'])
        pos_on_e = np.array([-l_x, 0, l_x])
        for i in range(3):
            position = h2e(e.geometry['center']) + pos_on_e[i] * h2e(e.geometry['e_y'])
            vec_pos_foc = - position + h2e(focalpoint)
            vec_pos_foc = vec_pos_foc / np.linalg.norm(vec_pos_foc)
            vec_pos_ref_point = - position + h2e(ref_point)
            vec_pos_ref_point = vec_pos_ref_point / np.linalg.norm(vec_pos_ref_point)

            theta_needed = np.arccos(np.dot(vec_pos_foc, vec_pos_ref_point))
            d_needed[i] = np.abs(order) * wave.to(u.mm).value / np.sin(theta_needed)
        e._d_needed = d_needed
        e._chirp = (d_needed[2] - d_needed[0]) / (2 * l_x)
        def func(intercoos):
            return intercoos[:, 0] * e._chirp + 2e-4
        e._d = func


In [ ]:
from scipy.optimize import brent
from marxs.math.utils import e2h    


def chirp_flat_grating_numerical(conf, gas, order, wave):
    '''
    Parameters
    ----------
    wave : `astropy.quantity.Quantity`
    '''
    focalpoint, ref_point = find_where_ref_ray_should_go(conf, order, wave)

    def mock_order(x, y, z):
        return np.array([0, order]), np.ones(2)
        
    for e in gas.elements:

        l_x = np.linalg.norm(e.geometry['v_y'])
        # Place just inside the edges. Right on the edges might miss due to numerics
        pos_on_e = np.array([-0.999 * l_x, 0, 0.999 * l_x])
        
        e.order_selector_backup = e.order_selector
        e.order_selector = mock_order
        d_needed = np.zeros(len(pos_on_e))
        
        for i in range(len(pos_on_e)):
            position = h2e(e.geometry['center']) + pos_on_e[i] * h2e(e.geometry['e_y'])
            vec_pos_foc = - position + h2e(focalpoint)
            vec_pos_foc = vec_pos_foc / np.linalg.norm(vec_pos_foc)
            vec_pos_ref_point = - position + h2e(ref_point)
            vec_pos_ref_point = vec_pos_ref_point / np.linalg.norm(vec_pos_ref_point)
            photons = Table({'pos': np.tile(e2h(position, 1), (2, 1)),
                     'dir': np.tile(e2h(vec_pos_foc, 0), (2, 1)),
                     'energy': np.ones(2) * wave.to(u.keV, equivalencies=u.spectral()),
                     'polarization': np.tile([0.,1.,0.,0.], (2, 1)),
                     'probability': np.ones(2),
                     })
            # Move back a little
            photons['pos'] = photons['pos'] - photons['dir']
        
    
            def theta(d):
                e._d = d
                p = e(photons.copy())
                a = vec_pos_ref_point[:3]
                b = p['dir'][1, :3]
                return np.arccos(np.dot(a, b))
            
            d_needed[i] = brent(theta, brack=(0.0001, 0.0003))            

        e.order_selector = e.order_selector_backup                
        e._d_needed = d_needed
        e._chirp = (d_needed[2] - d_needed[0]) / (2 * l_x)
        def func(intercoos):
            return intercoos[:, 0] * e._chirp + d_needed[1]
        e._d = func


In [ ]:
for e in lynx5050.elements[2].elements:
        e._chirp = (e._d_needed[2] - e._d_needed[0]) / (2 * l_x)
        def func(intercoos):
            return intercoos[:, 0] * e._chirp + e._d_needed[1]
        e._d = func


In [ ]:
def mock_order(x, y, z):
        return np.array([0, order]), np.ones(2)
        
order = -6
d_needed = np.zeros(3)
l_x = np.linalg.norm(e.geometry['v_y'])
# Place just inside the edges. Right on the edges might miss due to numerics
pos_on_e = np.array([-0.999 * l_x, 0, 0.999 * l_x])
        
e.order_selector_backup = e.order_selector
e.order_selector = mock_order
        
i = 2
position = h2e(e.geometry['center']) + pos_on_e[i] * h2e(e.geometry['e_y'])
vec_pos_foc = - position + h2e(focalpoint)
vec_pos_foc = vec_pos_foc / np.linalg.norm(vec_pos_foc)
vec_pos_ref_point = - position + h2e(ref_point)
vec_pos_ref_point = vec_pos_ref_point / np.linalg.norm(vec_pos_ref_point)
            
theta_needed = np.arccos(np.dot(vec_pos_ref_point, vec_pos_foc))

photons = Table({'pos': np.tile(e2h(position, 1), (2, 1)),
                     'dir': np.tile(e2h(vec_pos_foc, 0), (2, 1)),
                     'energy': np.ones(2) * wave.to(u.keV, equivalencies=u.spectral()),
                     'polarization': np.tile([0.,1.,0.,0.], (2, 1)),
                     'probability': np.ones(2),
                     })
# Move back a little
photons['pos'] = photons['pos'] - photons['dir']
        
    
def theta(d):
                e._d = d
                p = e(photons.copy())
                return np.arccos(np.dot(vec_pos_ref_point, p['dir'][1, :3]))
            
brent(theta, brack=(0.001, 0.003))
            


In [ ]:
conf_chirp = copy.copy(lynx.conf)
conf_chirp['grating_size'] = np.array([50, 50])
#l_chirp = lynx.PerfectLynx(conf=conf_chirp)
chirp_flat_grating_numerical(conf_chirp, lynx5050.elements[2], -6, (0.6 * u.keV).to(u.Angstrom, 
                                                                                    equivalencies=u.spectral()))

In [ ]:
p_chirp = run_monoenergetic_simulation(lynx5050, single_en)
p_chirp = join(p_chirp, facettab5050)

In [ ]:
for i in range(len(lynx5050.elements[2].elements)):
    if lynx5050.elements[2].elements[i].order_selector != lynx5050.elements[2].elements[0].order_selector:
        print(i)

In [ ]:
lynx5050.elements[2].elements[2420].order_selector = lynx5050.elements[2].elements[10].order_selector

In [ ]:
for e in lynx5050.elements[2].elements[::4]:
    plt.plot(e.geometry['center'][1], e._d_needed[2], '.')

In [ ]:
for e in lynx5050.elements[2].elements[::4]:
    plt.plot(e.geometry['center'][1], e._d_needed[0], '.')

In [ ]:
plt.plot(pdef['pos'][:, 1], pdef['pos'][:, 2], '.')

In [ ]:
pdef.label = '50*20 mm - offcenter'
p_scat.label = '50*20 mm - scatter'
p5050.label = '50*50 mm - offcenter'
#pb_scat.label = '50*20 - bend'
#p_chirp.label = '50*50 mm - chirp'
photons = [pdef, p_scat, p5050]

In [ ]:
conf_scat_small = copy.copy(lynx.conf)
conf_scat_small['grating_size'] = [50, 20]
conf_scat_small['inplanescatter'] = 3.5e-6
conf_scat_small['perpplanescatter'] = .7e-6
l_scat_small = lynx.PerfectLynx(conf=conf_scat_small)
p_scat_small = run_monoenergetic_simulation(l_scat_small, single_en)
p_scat_small = join(p_scat_small, facettab_small)

In [ ]:
len(facettabdef), len(facettab5050)

In [ ]:
out = plt.hist(np.rad2deg(pdef['blaze']), bins=np.linspace(1.5, 1.7, 50), label='Flat grating', lw=2)
out = plt.hist(np.rad2deg(pb_scat['blaze']), bins=np.linspace(1.5, 1.7, 50), histtype='step', label='Curved grating', lw=2)
plt.xlabel('Blaze angle [deg]')
plt.ylabel('Number of photons')
plt.savefig('/Users/hamogu/MITDropbox/my_talks/18_SPIE_Lynx/talk-lynxXGS-SPIE18/images/blaze.png', dpi=300)
plt.savefig('/Users/hamogu/MITDropbox/my_talks/18_SPIE_Lynx/blaze.pdf', bbox_inches='tight')

In [ ]:
from matplotlib.ticker import NullFormatter

ind = (photons['order'] == 0) & (np.abs(photons['proj_x'] < 10))
pgroups = photons[ind].group_by('order')
x = [p['proj_x'] for p in pgroups.groups]
y = [p['proj_y'] for p in pgroups.groups]

nullfmt = NullFormatter()         # no labels

# definitions for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
bottom_h = left_h = left + width + 0.02

rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom_h, width, 0.2]
rect_histy = [left_h, bottom, 0.2, height]

# start with a rectangular Figure
plt.figure(1, figsize=(5, 5))

axScatter = plt.axes(rect_scatter)
axHistx = plt.axes(rect_histx)
axHisty = plt.axes(rect_histy)

# no labels
axHistx.xaxis.set_major_formatter(nullfmt)
axHisty.yaxis.set_major_formatter(nullfmt)

# the scatter plot:
axScatter.scatter(x, y)

# now determine nice limits by hand:
binwidth = 0.0025
xymax = np.max([np.max(np.fabs(x)), np.max(np.fabs(y))])
lim = (int(xymax/binwidth) + 1) * binwidth

axScatter.set_xlim((-lim, lim))
axScatter.set_ylim((-lim, lim))

bins = np.arange(-lim, lim + binwidth, binwidth)
histx = axHistx.hist(x, bins=bins, stacked=True)
histy = axHisty.hist(y, bins=bins, orientation='horizontal')

axHistx.set_xlim(axScatter.get_xlim())
axHisty.set_ylim(axScatter.get_ylim())

axScatter.set_xlabel('dispersion direction [mm]')
axScatter.set_ylabel('crossdispersion direction [mm]')

In [ ]:
def hpd(x, y):
    '''Simple estimate for the half-power-diameter
    '''
    r = np.sqrt((x - x.mean())**2 + (y - y.mean())**2)
    return np.median(r)

In [ ]:
p0 = p_scat[p_scat['order'] == 0]
print('Estimate for HPD [in mm]:', hpd(p0['proj_x'], p0['proj_y']))
print('0.5 arcsec correspond to {} mm.'.format(np.deg2rad(0.5/3600.) * 10000.))

These numbers indicates that we set up our mirror correctly such that the HPD is close to 0.5 arcsec. Looking at the histrogram in the plot, the shape of the PSF is also roughly Gaussian, so that thsese simulations can be compared with other efforts for the Lynx development, where similar PSFs are used.

## A simulation at 0.5 keV (= 2.4 nm = 24 Angstrom)

First, we look at a simulation at one specific energy and see how the diffracted orders look on the detector and what we can learn from sub-aperturing. Later, we will repeat this analysis for a grid of photon energies, but it is useful to look in a little more detail for a single energy first to understand what is going on.

The convention that MARXS, our ray-trace code, uses for CAT gratings is to label the diffraction orders with negative numbers, so this is what we use in the following figures.

In [ ]:
orders = order_selector_Si.orders

labels = ['fiducial', 'small', 'scatt', 'scat_small', 'bend_scat', 'bend_scat_ind']

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
for i, p in enumerate([pdef, p_scat, p5050]): #photons, p_small, p_scat, p_scat_small, pb_scat, pbn_scat]):
    n_p = np.array([p[p['order'] == o]['probability'].sum() for o in orders])
    aeff = lynxdef.elements[0].area.to(u.cm**2) / 1e5 * n_p
    ax.plot(orders, aeff, label=labels[i])
    
ax.legend()
ax.set_xlim(-9,4)

In [ ]:
photons = photons[photons['probability'] > 0]
plt.scatter(photons['detcircpix_x'], photons['detcircpix_y'], c=photons['order'])
plt.colorbar(label='Diffraction order')
plt.xlabel('Dispersion direction in focal plane [mm]')
plt.ylabel('Cross-dispersion direction [mm]')

Figure above: Position of photons projected into the focal plane. Dispersion goes from left to right. Note that the scale of the x and y axis is *very* different!

We simuluate a Rowland spectrometer. As such, the CCD detectors are **not** in the focal plane, instead they follow the curved surface of the Rowland torus. This optimized for the spectral focus, i.e. it makes the orders narrow in dispersion direction, but at the cost of a wider distribution in the cross-dispersion direction. Only for the zeroth order (red, leftmost dot) do the imaging focus and the spectral focus agree and thus this order is circular as shown in the plot above. For CAT gratings, it is useful to use a *tilted* Rowland torus, which intersects the focal plane a second time, thus there is a second place (around 700 mm for the parameters chosen here) where the dispersed order is small in dispersion and cross-dispersion direction. This leads the fish-shaped distribution seen in the plot above. We now look at one of those orders in more detail.

In [ ]:
cmap = plt.get_cmap('hsv')

In [ ]:

fig = plt.figure(figsize=(15, 6))
ax1 = fig.add_subplot(121, aspect='equal')
ax1.scatter(facetpos_small[:, 1, 3], facetpos_small[:, 2, 3], c=np.rad2deg(facettab_small['facet_ang']),
            s=5, marker='s', cmap=cmap)
ax2 = fig.add_subplot(122)
ind = (p_small['order'] == -7) & (p_small['proj_x'] > 0)
scat = ax2.scatter(p_small['proj_x'][ind], p_small['proj_y'][ind], 
            c=np.rad2deg(p_small['facet_ang'])[ind], cmap=cmap)
#plt.xlim([-0.1, 0.1])
ax2.set_ylim([-1, 1])
plt.colorbar(scat, ax=ax2, label='Angle of grating facet')
ax2.set_xlabel('Dispersion direction in focal plane [mm]')
ax2.set_ylabel('Cross-dispersion direction [mm]')
ax1.set_xlabel('Distance from optical axis [mm]')
ax1.set_ylabel('Distance from optical axis [mm]')

In [ ]:
photons.colnames

In [ ]:
photons = pdef
fig = plt.figure(figsize=(9,4))
ax1 = fig.add_subplot(131, aspect='equal')
ax1.scatter(facetposdef[:, 1, 3], facetposdef[:, 2, 3], c=np.rad2deg(facettabdef['facet_ang']),
            s=10, marker='s', cmap=cmap)
ax2 = fig.add_subplot(132)
ind = (photons['order'] == -6) & (photons['proj_x'] > 0)
scat = ax2.scatter(photons['proj_x'][ind], photons['proj_y'][ind], 
            c=np.rad2deg(photons['facet_ang'])[ind], cmap=cmap)
#plt.xlim([-0.1, 0.1])
#plt.ylim([-0.1, 0.1])
ax3 = fig.add_subplot(133)
ind = (p_chirp['order'] == -6) & (p_chirp['proj_x'] > 0)
scat = ax3.scatter(p_chirp['proj_x'][ind], p_chirp['proj_y'][ind], 
            c=np.rad2deg(p_chirp['facet_ang'])[ind], cmap=cmap)

plt.colorbar(scat, ax=ax2, label='Angle of grating facet')
ax2.set_xlabel('Dispersion direction in focal plane [mm]')
ax2.set_ylabel('Cross-dispersion direction [mm]')
ax1.set_xlabel('Distance from optical axis [mm]')
ax1.set_ylabel('Distance from optical axis [mm]')
#fig.savefig('/Users/hamogu/MITDropbox/my_poster/18_SPIE_Lynx/talk/images/orders.png', dpi=300, bbox_inches='tight')

In [ ]:
labels = ['fiducial', 'small', 'scatt', 'scat_small']

fig = plt.figure(figsize=(10, 10))
for i, p in enumerate(photons):
    ax = fig.add_subplot(2, 2, i + 2)
    ind = (p['order'] == -6) 
    scat = ax.scatter(p['proj_x'][ind], p['proj_y'][ind], 
            c=np.rad2deg(p['facet_ang'])[ind], cmap=cmap)
    #plt.colorbar(scat, ax=ax2, label='Angle of grating facet')
    ax.set_xlabel('Dispersion direction in focal plane [mm]')
    ax.set_ylabel('Cross-dispersion direction in focal plane [mm]') 
    ax.set_title(p.label)
    ax.set_xlim([595.6, 596.3])

ax1 = fig.add_subplot(221, aspect='equal')
ax1.scatter(facetposdef[:, 1, 3], facetposdef[:, 2, 3], c=np.rad2deg(facettabdef['facet_ang']),
            s=10, marker='s', cmap=cmap)
ax1.set_title('Gratings in aperture')
ax1.set_xlabel('Grating position [mm] along dispersion')
ax1.set_ylabel('Grating position [mm] along cross-dispersion')

fig.savefig('/Users/hamogu/MITDropbox/my_talks/18_SPIE_Lynx/talk-lynxXGS-SPIE18/images/orders.png', dpi=300)
fig.savefig('/Users/hamogu/MITDropbox/my_talks/18_SPIE_Lynx/orders.pdf', bbox_inches='tight')

The left plot shows the arrangement of the grating facets looking from end of the mirrors towards the focal plane. The dispersion direction is from left to right and cross-dispersion from top to bottom. There is no special meaning in that gap at the middle right. It's just where I started distributing the grating facets. The gap is just too small to place another set of facets so it's left empty. The distribution of facets can be improved to reduce the uncovered area in between, but that this stage it is not useful to spend to much detailed work on that. Note that the coloring schme is different than in the 3D view on the interactive 3d website. Facets are colored according to the angle that the facet center has towards the positive dispersion direction.

The plot on the right shows a spefic grating order on the detector (order 7). The detector is cylindrical following the Rowland circle. Intersection positions with this detecor are the projected on the focal plane so that I can properly display them in a two-dimensional plot. Individual photons are colored according to the grating facet that they passed through. There are several things we can learn from this plot: First, note that the x and y axis are not to scale, this really is much longer in cross-dispersion direction (top to bottom) and narrow in the dispersion direction (left to right). Second, the aboslute position along the dispersion direction is 75 cm from the focal point, so the dispersion angle is about 5 deg - significantly more than in Chandra or XMM-Newton. Thus, abberations that do not need to be considered for Chandra or XMM-Newton can be important here. 

So, we can now look at the distribution of colors and we see immediately that the distribution is not homogeneous. Photons that went through the cyan, green, and yellow colored gratings are distributed a narrower than the blue and red points. Thus, sub-aperturing *can* defintely increase the spectral resolving power. However, the pattern of the sub-aperturing is completely different than the "normal" pattern. Typically, sub-aperturing is done using only an area along the cross-dispersion direction (here the orange and light blue gratings). In contrast, here, we achieve the best spectral resolving power by using only gratings that are located along the forward dispersion direction.

This might be a surprising result at first glance, but thinking back to the assumptions that went into this simulation, it might seem a little more sensible. We set up the PSF such that there is no difference between in-plane and out-of-plane scatter and thus there is nothing special about the cross-dispersion direction. Instead the shape of the PSF is dominated by other things. Now, recall that this order is located about 75 cm from the optical axis; at the same time the facets go from -60 cm (red and blue) to +60 cm (light green). There is a consideralbe path length difference between those photons. The light green facets are located essentially "directly above" the order, while the red/blue ones are 1.5 m to the side; so it should not come a a complete surprise that the abberations differ between those two groups.

In the following, I show how much better the spectral resolving power can be for different opening angles of the subaperture. That angle measures the angle between the center of a facet and the positive horizonal axis (the gap in the left iamge above). *There is no second mirrored sector*. For example, a subaperture angle of 30 degrees includes the cyan and light green gratings, 90 deg included lightblue, cyan, green, yellow, and orange, and only the full aperture (suaperture angle 180 deg) includes all gratings, even the dark red and dark blue ones.

In [ ]:
subaperangle = np.linspace(0, np.pi, 7)[1:]

def res_power_angle(photons, subaperangle, ang_0=0):
    resolvingpower = np.zeros((len(subaperangle), len(orders)))
    aeff_per_order = np.zeros_like(resolvingpower)
    for i, ang in enumerate(subaperangle):
        ind = np.abs(np.abs(photons['facet_ang']) - ang_0) < ang
        res, width, pos = resolvingpower_from_photonlist(photons[ind], orders, zeropos=0)
        resolvingpower[i, :] = res
        aeff_per_order[i, :] = [photons['probability'][ind & (photons['order'] == o)].sum() for o in orders]
    aeff_per_order *= lynxdef.elements[0].area.to(u.cm**2) / photons.meta['EXPOSURE'][0]
    return resolvingpower, aeff_per_order

In [ ]:
subaperangle = np.linspace(0, np.pi, 7)[1:]
tsubaperangle = np.linspace(0, np.pi/2, 7)[1:]

for p in [pdef, p_scat, p5050]:
    trespow, taeff = res_power_angle(p, tsubaperangle, np.pi/2)
    p.trespow = trespow
    p.taeff = taeff
    respow, aeff = res_power_angle(p, subaperangle)
    p.respow = respow
    p.aeff = aeff

In [ ]:
p = p5050

fig = plt.figure() 

for i, ang in enumerate(tsubaperangle):
    plt.plot(orders, 
             p.trespow[i, :], 
             label='{:3.0f}'.format(np.rad2deg(ang)))
plt.legend(title='Subaperture\nangle [deg]', loc='upper left')
plt.ylabel('Resolving power')
plt.xlabel('Grating order')
plt.gca().invert_xaxis()

In [ ]:
p5 = photons[0][photons[0]["order"] == -4]

In [ ]:
plt.plot(p5['proj_x'], p5['proj_y'],'.')

In [ ]:
p.trespow[:, i]

In [ ]:
pdef.trespow[:, 10]

In [ ]:
i =  (orders == -6)
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)

for p in photons:
    ax.plot(np.average(p.trespow[:, i], weights=p.taeff[:, i], axis=1), 
             2* p.taeff[:, i].sum(axis=1), label=p.label)
# 2 times is cheat here because order -5 falls in a chip gap!
ax.legend()
ax.set_xlabel('Resolving power')
ax.set_ylabel('Effective area [cm$^2$]')
fig.savefig('/Users/hamogu/MITDropbox/my_talks/18_SPIE_Lynx/talk-lynxXGS-SPIE18/images/traderaeff.png', dpi=300, bbox_inches='tight')
fig.savefig('/Users/hamogu/MITDropbox/my_talks/18_SPIE_Lynx/traderaeff.pdf', bbox_inches='tight')

Resolving power depending on the grating order (or diffraction angle). Sub-aperturing does little for low orders located close to the optical axis but can improve the resolving power up to a factor of three for higher orders. In practice though, only some orders receive a sufficient number of photons to do spectral analysis. Thus, we can take a weighted average of the resolving power for all orders where the resolving power for each order is weighted by the number of photons it receives.

In [ ]:
from scipy import optimize

e = perfectlynx.elements[2].elements[200]
rowland = lynx.conf['rowland']

def make_opt_func(pos, dir, rowland):
    def f(x):
        return rowland.quartic(pos + x * dir)
    return f

pos_on_e = np.meshgrid(np.linspace(-1, 1, 11), np.linspace(-1, 1, 11))


In [ ]:
e = perfectlynx.elements[2].elements[2087]
print(e.geometry['v_y'][:3])
dist = np.zeros((11, 11))
for i in range(11):
    for j in range(11):
        pos = e.geometry['center'][:3] + pos_on_e[0][i, j] * e.geometry['v_y'][:3] + pos_on_e[1][i, j] * e.geometry['v_z'][:3] 
        dist[i, j] = optimize.brentq(make_opt_func(pos, e.geometry['e_x'][:3], rowland), -100, 100)
plt.imshow(dist)
plt.colorbar()

In [ ]:
e = perfectlynx.elements[2].elements[0]
dist = np.zeros((11, 11))
print(e.geometry['v_y'][:3])
for i in range(11):
    for j in range(11):
        pos = e.geometry['center'][:3] + pos_on_e[0][i, j] * e.geometry['v_y'][:3] + pos_on_e[1][i, j] * e.geometry['v_z'][:3] 
        dist[i, j] = optimize.brentq(make_opt_func(pos, e.geometry['e_x'][:3], rowland), -100, 100)
plt.imshow(dist)
plt.colorbar()

In [ ]:
e = perfectlynx.elements[2].elements[1047]
dist = np.zeros((11, 11))
print(e.geometry['v_y'][:3])
for i in range(11):
    for j in range(11):
        pos = e.geometry['center'][:3] + pos_on_e[0][i, j] * e.geometry['v_y'][:3] + pos_on_e[1][i, j] * e.geometry['v_z'][:3] 
        dist[i, j] = optimize.brentq(make_opt_func(pos, e.geometry['e_x'][:3], rowland), -100, 100)
plt.imshow(dist)
plt.colorbar()

In [ ]:
e = perfectlynx_b_scat.elements[2].elements[20]
pos = e.geometry.parametric_surface(z=np.linspace(-1, 1, 11))
direc = e.geometry.get_local_euklid_bases(np.array([[0, 0]]))
dist = np.zeros(pos.shape[:2])
for i in range(pos.shape[0]):
    for j in range(pos.shape[1]):
        dist[i, j] = optimize.brentq(make_opt_func(pos[i, j, :3], direc[2][0, :3], rowland), -5, 5)
plt.imshow(dist)
plt.colorbar()

In [ ]:
facettab[1000: 1050]

In [ ]:
#resolvingpower = np.ma.masked_invalid(resolvingpower)
#np.ma.average(resolvingpower, axis=1, 
#              weights=order_selector_Si.proba
#            abilities([0.5], [0], [lynx.conf['blazeang']])[1].flatten())

## Simulations for different energies

Now, I run simulations with the same set-up as above for different input energies. The goal is not to produce a fine grid, but rather to set a few goalposts to see how the effect of sub-aperturing changes with energy.

In [ ]:
subaperangle = np.linspace(0, np.pi, 7)[1:]
energy = np.arange(0.3, 1.8, .02) * u.keV

In [ ]:
wavegrid = energy.to(u.nm, equivalencies=u.spectral())

In [ ]:
phot_en = []

for i, e in enumerate(energy):
    print(e)
    n = 1e4
    if e.value > 1.25:
        n=1e5
    
    phot = run_monoenergetic_simulation(lynxdef, e.value, 1e4)
    phot = join(phot, facettabdef)
    phot_en.append(phot)


In [ ]:
for p in phot_en:
    ind = p['CCD_ID'] >= 0
    trespow, taeff = res_power_angle(p[ind], tsubaperangle, np.pi/2)
    p.trespow = trespow
    p.taeff = taeff
    respow, aeff = res_power_angle(p[ind], subaperangle)
    p.respow = respow
    p.aeff = aeff

In [ ]:
from matplotlib.ticker import MaxNLocator

fig = plt.figure(figsize=(15, 15))
lenen = len(energy)
indorders = np.arange(4, 16)
lenord = indorders.shape[0]

axes = np.empty((lenen, lenord), dtype=object)

for i in range(len(energy)):
    axes[i, 0] = fig.add_subplot(lenen, lenord, lenord * i + 1)
    axes[i, 0].set_ylabel(str(energy[i]))
    plt.setp(axes[i, 0].get_yticklabels(), visible=False)
    for j in range(1, len(indorders)):
        axes[i, j] = fig.add_subplot(lenen, lenord, lenord * i + j + 1) #, sharey=axes[i, 0])
        plt.setp(axes[i, j].get_yticklabels(), visible=False)
        
for i in range(len(energy)):
    phot = phot_en[i]
    for j, o in enumerate(indorders):      
        order = orders[o]
        ind = (phot['order'] == order) #& (np.abs(phot['proj_y']) < 10)
        axes[i, j].scatter(phot['proj_x'][ind], phot['proj_y'][ind], 
                           c=np.rad2deg(phot['facet_ang'])[ind],
                           edgecolors='none', cmap=cmap)
        

for j, o in enumerate(indorders):
    axes[0, j].set_title('order {}'.format(orders[o]))
    
#axes[-1, 5].set_xlabel('position on dispersion direction [mm]. Note that the scale differs for every plot.')
    
fig.subplots_adjust(wspace=0)

The plot above shows the PSF for different grating orders and energies. The energy for each row is listed on the left, and then the PSF is shown for order 0, -1, -2 ,... from left to right. THe x-axis in eahc panel has a differnet scaling and shows the position in dispersion direction (in mm) measured from the optical axis. Dots are colored according to the angle of the facet that the photon went through using the same color scale as above. For each order, the y axis is scaled differently. As shown in the fish-shaped plot above, the width in the y direction ranges from 0.02 mm for order 0 to a a few mm for higher orders. Using a the same scaling in y direction for all orders in a row would hide several of the details for the PSFs I'm going to discuss now, but keep in mind that the scaling of x and y axis is not the same. For exmaple, all zeroth orders are really circular and not elliptical.

Let me know highlight a few insteresting aspects about the use of subaperturing for different energies and orders. This discussion here is qualitatively and I try to explain why things look the way they do and later I will show the actual number for the resolving power. Looking at the zeroth order for any energy, all the colors are well mixed in the plot and sub-aperturing will not do anything to change that. This is simply the result of our setup, where the scattering of the mirror is the same in every direction. At 0.3 keV most of the photons are found in order -3 and -4. In both cases, we see that sub-aperturing as discussed in detail above (selecting the angles for cyan, green, and yellow) would significantly narrow the order and thus improve the spectral resolving power. It is interesting to note that the color scheme seems flipped between order -3 and -4. In order -3 the blue-ish photons are found on the top and the reddish photons at the bottom, in order -4 that is reversed. The Rowland torus optimizes the focussing in the dispersion direction, but the focus in the cross-dispersion direction is different. In one case the detector is located below the focus in cross-dispersion direction (so in y direction photons have passed through the focus and spread out again), in the other case it is above the cross-dispersion focus. This flip happens around 62 cm, where the 7 order of 0.6 keV photons is located. Photons of higher energy (and thus lower wavelength) are not dispersed this far out and thus there is no flip in the plots for 1.0, 1.4, and 1.8 keV.

In general, for higher energies where the grating orders are located closer to the optical axis, the distribution of the photons in each order depends less on the angle of the facet that they went through and thus sub-aperturing cannot do much to improve the spectral resolving power. The simulations for 1.4 and 1.8 keV photons show this. At these energies, the efficiency also peaks at much lower orders and the relevant signal is only a few cm from the optical axis (for example, order -2 for 1.8 keV is at 59 mm = 5.9 cm. 

At 1 keV, a significant number of photons are seen in very low orders (-1 to -3) where sub-aperturing is not relevant, but then again there are many photons in order -9. If Lynx uses sub-aperturing, then one could analyze the photons in order 9 for high-resolution work, if instead a larger number of photons is required for the science goal, but spectral resolving power is not crucial, then the observer could include the lower orders in the analysis. However, for simplicity, I will just average the resolving power for all orders when I show plots of resolving power vs sub-aperturing angle below.

I want to point out one more feature: While the signal of the cyan, green, and yellow photons is generally is approximately centered on the same position as the wider distribution for the red and blue photons, this is not the case around 40-50 cm from the optical axis (order -2 for 0.3 keV; order -5 for 0.6 keV; order -9 for 1 keV).


In [ ]:
resolvingpower_en = np.zeros((len(subaperangle), len(energy)))

for i, e in enumerate(energy):
    phot = phot_en[i]
    resolvingpower = np.zeros((len(subaperangle), len(orders)))
    for j, ang in enumerate(subaperangle):
        ind = phot['CCD_ID'] >=0
        res, width, pos = resolvingpower_from_photonlist(phot[np.abs(phot['facet_ang']) < ang],
                                                         orders, col='detcirc_phi')
        resolvingpower[j, :] = res
        
    resolvingpower = np.ma.masked_invalid(resolvingpower)
    # Mask out the zeros order which always has resolving power 0 
    resolvingpower[:, 0] = np.ma.masked
    res = np.ma.average(resolvingpower, axis=1, 
                        weights=order_selector_Si.probabilities([e], [0], [lynx.conf['blazeang']])[1].flatten())
    resolvingpower_en[:, i] = res
        

In [ ]:
for i, ang in enumerate(subaperangle):
    plt.plot(energy, resolvingpower_en[i, :], label='{:3.0f}'.format(np.rad2deg(ang)))
plt.legend(title='Subaperture\nangle [deg]')
plt.ylabel('Resolving power')
plt.xlabel('Photon energy [keV]')

This plot shows the spectral resolving power vs energy for different sub-aperturing angles. When calculating the average, different orders are weighted according to the number of photons they receive. Sub-aperturing will not increase the resolving power for high energies, but sacrificing 2/3 of the effective area in a 60 degree sub-aperture angle would increase the resolving power at low energies by almost a factor of three.

In [ ]:
en_trespos = np.stack([p.trespow for p in phot_en])
en_taeff = np.stack([p.taeff for p in phot_en])
en_trespos.shape

In [ ]:
len(energy)

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)

for i, ang in enumerate(tsubaperangle):
    ax.plot(wavegrid, en_taeff[:, i, :].sum(axis=1), label='{:3.0f}'.format(np.rad2deg(ang)))
#ax.legend(title='Subaperture\nangle [deg]')
ax.set_ylabel('Effective Area [cm$^2$]')
ax.set_xlabel('wavelength [nm]')

fig.savefig('/Users/hamogu/MITDropbox/my_talks/18_SPIE_Lynx/talk-lynxXGS-SPIE18/images/aeff_en.png', 
            dpi=300, bbox_inches='tight')
fig.savefig('/Users/hamogu/MITDropbox/my_talks/18_SPIE_Lynx/aeff_en.pdf', bbox_inches='tight')

In [ ]:
# There are Nans in there which srew up the average
en_trespos[np.isnan(en_trespos)] = 0

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)

for i, ang in enumerate(tsubaperangle):
    ax.plot(wavegrid, np.average(en_trespos[:, i, :], axis=1, weights=en_taeff[:, i,:]), 
             label='{:3.0f}'.format(np.rad2deg(ang)))
ax.legend(title='Subaperture\nangle [deg]')
ax.set_ylabel('Resolving power')
ax.set_xlabel('wavelength [nm]')

fig.savefig('/Users/hamogu/MITDropbox/my_talks/18_SPIE_Lynx/talk-lynxXGS-SPIE18/images/rew_en.png', dpi=300, bbox_inches='tight')
fig.savefig('/Users/hamogu/MITDropbox/my_talks/18_SPIE_Lynx/rew_en.pdf', bbox_inches='tight')

In [ ]:
en_trespos[1, i, :]

In [ ]:
en_taeff[1, i, :]

In [ ]:
for i, e in enumerate(energy):
    phot = phot_en[i]
    resolvingpower = np.zeros((len(subaperangle), len(orders)))
    for j, ang in enumerate(subaperangle):
        res, width, pos = resolvingpower_from_photonlist(phot[np.abs(phot['facet_ang']) < ang],
                                                         orders, col='detcirc_phi')
        resolvingpower[j, :] = res
        
    resolvingpower = np.ma.masked_invalid(resolvingpower)
    # Mask out the zeros order which always has resolving power 0 
    resolvingpower[:, 0] = np.ma.masked
    res = np.ma.average(resolvingpower, axis=1, 
                        weights=order_selector_Si.probabilities([e], [0], [lynx.conf['blazeang']])[1].flatten())
    resolvingpower_en[:, i] = res
        

## Future work

Much remains to be done to study this in more detail. There are several parameters in the simulation that I fixed to certain values based on experience with simulating other observatories, but that remain to be studied in the context of Lynx. An incomplete list is:

- **mirror PSF** Does this analysis hold if the mirror shells are aligned better so that scattering in the plane of reflection is actually different from the plane of reflection? This might lead to a more conventional pattern of the sub-aperturing.
- **Facet size** Flat grating facets of finite size always deviate from the Rowland torus. The facets here are square with 50 mm sides. How does the resolving power improve with smaller facets?
- **Torus tilt** The torus is tilted by a little more than twice the facet blaze angle. There are two free parameters in here which Heilmann et al. 2010 call "hinge points". I don't think that the exact numbers are critical for the answer, but that should be checked.

In [ ]:
for e in lynxdef.elements[6].elements:
    print(e.geometry['center'])

In [ ]:
for e in lynxdef.elements[6].elements:
    print(e.geometry['center'] - e.geometry['v_y'])

In [ ]:
for e in lynxdef.elements[6].elements:
    print(e.geometry['center'] + e.geometry['v_y'])

In [ ]:
row = lynx.conf['rowland']

In [ ]:
row['center']

In [ ]:
row['e_x']

In [ ]:
row['e_y']

In [ ]:
row['e_z']